In [1]:
import pandas as pd

def read_csv_column(csv_path, column_name):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    return df[column_name].tolist()


def read_csv_with_weight(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)

    # 转发数,评论数,点赞数
    comments = df['评论数'].tolist()
    reposts = df['转发数'].tolist()
    likes = df['点赞数'].tolist()
    weibos = df['微博正文'].tolist()
    results = []

    # for row in range(len(weibos)):
    #     if comments[row] + reposts[row] + likes[row] > 10:
    #         results.append(weibos[row])

    # for row in range(len(weibos)):
    #     times = 1 + int(reposts[row]/100) + int(comments[row]/500) + int(likes[row]/1000)
    #     for time in range(times):
    #         results.append(weibos[row])
    results = weibos

    new_list = []
    for result in results:
        result = result.split()
        new_list.append(result)

    return new_list

In [2]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaMulticore, CoherenceModel
from gensim.models.wrappers import LdaMallet


In [3]:
trump_content = read_csv_with_weight('./dataset/trump/trump01.csv')

In [4]:
dictionary = Dictionary(trump_content)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=3000)
bow_corpus = [dictionary.doc2bow(doc) for doc in trump_content]

In [5]:
tfidf = TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [6]:
lda_model_tfidf = LdaMulticore(corpus_tfidf, num_topics=30, id2word=dictionary, passes=10, workers=4)
lda_model_tfidf.save('./model/lda_models_tfidf_1.lda')

In [7]:
lda_model = LdaMulticore.load('./model/lda_models_tfidf_1.lda')
item = []
for j in range(len(trump_content)):
    item = item + trump_content[j]

bow_vector = dictionary.doc2bow(item)

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.15954715013504028	 Topic: 0.040*"伊朗" + 0.023*"美军" + 0.023*"伊拉克" + 0.018*"袭击" + 0.017*"导弹" + 0.015*"基地" + 0.012*"美国" + 0.009*"苏莱曼尼" + 0.009*"伤亡" + 0.008*"军事"
Score: 0.13253390789031982	 Topic: 0.017*"巴勒斯坦" + 0.010*"提供者" + 0.010*"经济" + 0.010*"黄金" + 0.009*"市场" + 0.009*"美国" + 0.008*"农历" + 0.008*"美元" + 0.006*"移民" + 0.006*"中国"
Score: 0.10597363859415054	 Topic: 0.021*"协议" + 0.017*"英国" + 0.015*"华为" + 0.013*"中国" + 0.012*"美国" + 0.011*"墨加" + 0.010*"政府" + 0.009*"5G" + 0.009*"贸易" + 0.007*"约翰逊"
Score: 0.05952197685837746	 Topic: 0.047*"弹劾" + 0.039*"参议院" + 0.035*"弹劾案" + 0.030*"审判" + 0.027*"审理" + 0.019*"正式" + 0.017*"塔利班" + 0.015*"民主党" + 0.013*"美联储" + 0.012*"参议员"
Score: 0.04680744558572769	 Topic: 0.074*"疫情" + 0.067*"中国" + 0.056*"冠状" + 0.055*"北约" + 0.052*"病毒" + 0.051*"建议" + 0.049*"名字" + 0.047*"新型" + 0.045*"提供" + 0.043*"肺炎"
Score: 0.04177666828036308	 Topic: 0.027*"战争" + 0.022*"世界" + 0.021*"伊朗" + 0.015*"大战" + 0.013*"美军" + 0.013*"空袭" + 0.013*"卫生" + 0.013*"阻止" + 0.011*"国家" + 0.010*"少将"
Score: 0.

In [8]:
for i in range(10):
    if i < 9:
        index = '0' + str(i + 1)
    else:
        index = str(i + 1)
    trump_content = read_csv_with_weight('./dataset/trump/trump' + index + '.csv')

    dictionary = Dictionary(trump_content)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=3000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in trump_content]
    tfidf = TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model_tfidf = LdaMulticore(corpus_tfidf, num_topics=30, id2word=dictionary, passes=10, workers=4)
    lda_model_tfidf.save('./model/lda_models_tfidf_' + index + '.lda')
    lda_model = LdaMulticore.load('./model/lda_models_tfidf_' + index + '.lda')
    item = []
    for j in range(len(trump_content)):
        item = item + trump_content[j]

    bow_vector = dictionary.doc2bow(item)

    print(index + '月')
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))
    print()

01月
Score: 0.14976796507835388	 Topic: 0.012*"关税" + 0.011*"经济" + 0.011*"提供者" + 0.011*"黄金" + 0.010*"市场" + 0.008*"美国" + 0.008*"贸易" + 0.007*"协议" + 0.007*"欧盟" + 0.006*"美元"
Score: 0.12331812083721161	 Topic: 0.040*"伊朗" + 0.031*"伊拉克" + 0.015*"美军" + 0.013*"空袭" + 0.011*"美国" + 0.011*"发动" + 0.011*"苏莱曼尼" + 0.010*"袭击" + 0.010*"身亡" + 0.010*"少将"
Score: 0.06710515916347504	 Topic: 0.034*"弹劾" + 0.029*"参议院" + 0.028*"弹劾案" + 0.026*"悼念" + 0.020*"审判" + 0.020*"审理" + 0.018*"去世" + 0.018*"奥巴马" + 0.017*"发文" + 0.014*"总统"
Score: 0.051942430436611176	 Topic: 0.066*"病毒" + 0.065*"疫情" + 0.055*"中国" + 0.050*"冠状" + 0.049*"新型" + 0.039*"提供" + 0.037*"肺炎" + 0.023*"航班" + 0.020*"武汉" + 0.019*"应对"
Score: 0.048879534006118774	 Topic: 0.039*"协定" + 0.031*"签署" + 0.029*"贸易" + 0.026*"协议" + 0.017*"墨加" + 0.015*"第一" + 0.014*"阶段" + 0.012*"北美" + 0.012*"加拿大" + 0.012*"美国"
Score: 0.04686972498893738	 Topic: 0.030*"巨星" + 0.018*"伊朗" + 0.010*"美国" + 0.009*"谴责" + 0.008*"局势" + 0.007*"美国人" + 0.007*"领袖" + 0.007*"佩洛西" + 0.007*"回应" + 0.007*"中东"
Score:

In [8]:
mallet_path = r'/Users/yueyifei/Tools/mallet-2.0.8/bin/mallet'

def coherence_values_computation(dictionary, corpus, texts, limit, start=2, step=3):
   coherence_values = []
   model_list = []
   for num_topics in range(start, limit, step):
      model = LdaMallet(
         mallet_path=mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word
      )
      model_list.append(model)
   coherence_model = CoherenceModel(
      model=model, texts=texts, dictionary=dictionary, coherence='c_v'
   )
   coherence_values.append(coherence_model.get_coherence())
return model_list, coherence_values

In [ ]:
model_list, coherence_values = coherence_values_computation (
   dictionary=id2word, corpus=corpus, texts=data_lemmatized,
   start=1, limit=50, step=8
)
limit=50
start=1
step=8
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()